<a href="https://colab.research.google.com/github/mary-ts/Neural-Computing/blob/tito/whoopsietito_of_xceptionGoodFruity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.layers as layers
from keras import Sequential
import keras.models
from keras.layers import *
from keras.models import * 
from keras import optimizers, applications
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

Dataset Paths

In [4]:
train_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/train/train"
test_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/test/test"

Creating training and test data

In [5]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split = 0.2
)
test_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

Importing data into generators

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (299,299),
    batch_size = 32,
    class_mode = 'categorical', 
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (299,299),
    batch_size = 32,
    shuffle=True,
    class_mode = 'categorical')

Found 16854 images belonging to 33 classes.
Found 0 images belonging to 0 classes.


Xception Model

In [7]:
model = applications.Xception(weights='imagenet', 
                              include_top=False, 
                              input_shape=(299, 299, 3))

for layer in model.layers[:-5]:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(.3)(x)
predictions = Dense(33, activation="softmax")(x)
model_final = Model(model.input, predictions)

model_final.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-5),loss='categorical_crossentropy',metrics=['accuracy'])
#model_final.summary()

83683744/83683744 [==============================] - 3s 0us/step


Xception


In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='loss', min_delta=0.0001, patience=7, verbose=2, mode='auto')
hist = model_final.fit_generator(generator=train_generator,                   
                                    steps_per_epoch=150,
                                    validation_data=test_generator,                    
                                    validation_steps=10,
                                    epochs=25,
                                    callbacks = [checkpoint, early],
                                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/25
150/150 [==============================] - ETA: 0s - loss: 1.7417 - accuracy: 0.5244 
Epoch 1: loss improved from inf to 1.74173, saving model to vgg16_1.h5
150/150 [==============================] - 2635s 17s/step - loss: 1.7417 - accuracy: 0.5244
Epoch 2/25
150/150 [==============================] - ETA: 0s - loss: 0.5777 - accuracy: 0.8169 
Epoch 2: loss improved from 1.74173 to 0.57774, saving model to vgg16_1.h5
150/150 [==============================] - 2529s 17s/step - loss: 0.5777 - accuracy: 0.8169
Epoch 3/25
150/150 [==============================] - ETA: 0s - loss: 0.3312 - accuracy: 0.8958 
Epoch 3: loss improved from 0.57774 to 0.33124, saving model to vgg16_1.h5
150/150 [==============================] - 2544s 17s/step - loss: 0.3312 - accuracy: 0.8958
Epoch 4/25
150/150 [==============================] - ETA: 0s - loss: 0.2358 - accuracy: 0.9223 
Epoch 4: loss improved from 0.33124 to 0.23584, saving model to vgg16_1.h5
150/150 [==============================]

In [ ]:
Y_pred = model.predict_generator(test_generator, 10 // 32+1)
y_pred = np.argmax(Y_pred, axiz=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Apple Braeburn',
'Apple Granny Smith',
'Apricot',
'Avocado',
'Banana',
'Blueberry',
'Cactus fruit',
'Cantaloupe',
'Cherry',
'Clementine',
'Corn',
'Cucumber RIpe',
'Grape blue',
'Kiwi',
'Lemon',
'Limes',
'Mango',
'Onion White',
'Orange',
'Papaya',
'Passion fruit',
'Peach',
'Pear',
'Pepper Green',
'Pepper Red',
'Pineapple',
'Plum',
'Pomegranate',
'Potato Red',
'Raspberry',
'Strawberry',
'Tomato',
'Watermelon']

Plot Xception

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["loss"])
plt.plot(hist.history["accuracy"])
#plt.plot(hist.history["val_loss"])
#plt.plot(hist.history["val_accuracy"])
plt.legend(["loss", "val_accuracy"])
plt.show()